Get the authors_df from mapvisualisation and embed them to create a network visualisation!

nomic api key: nk-A0J2eOiRGvYyglMPu8Kp9zEIYnsHHsyCOJFfyhjKlLc

open pkl with authors_df and adapt it to our wishes

In [2]:
import pickle as pkl
import pandas as pd

pd.set_option('display.max_colwidth',None) #shows the whole content not only the first x signs
authors_df = pd.read_pickle("./authors.pkl") #transfers content of pickle to dict
authors_df

,forename,lastname,affiliation,pubmedid,country,city,latitude,longitude,page_content,uid,Title,Published,Copyright Information,research
0,Adnan A,Al-Bdairi,"Infertility, Assisted Reproductive Technologies, Teba IVF and Genetic Center, Babylon, IRQ.",38738039,Iraq,Babylon,32.544713,44.431814,"Concerns about the SARS-CoV-2 pandemic's possible impact on sexual and reproductive health have grown significantly. In this narrative review, the latest research on the effects of SARS-CoV-2 infection on several facets of sexual and reproductive health is reviewed. The review starts initially by going through the possible consequences of SARS-CoV-2 on female menstrual cycles. The virus may interfere with the menstrual cycle, which could affect hormone synthesis and endometrial reactions, according to newly available evidence. Menstrual blood loss may be impacted by COVID-19's potential to influence endothelial cell function and systemic hemostasis. Hypothalamic amenorrhea may be brought on by severe COVID-19 disease. There is little research on this subject, although most women resume their regular menstrual cycles after 1-2 months of recuperation. The review also examines how SARS-CoV-2 infection may affect assisted reproductive technology (ART) treatments. There are few clinical data, although some research points to potential effects on embryo quality. Overall, ART results, however, did not materially change from the time before the epidemic. Obstetric problems are more likely when SARS-CoV-2 infection occurs during pregnancy, especially in the third trimester. Even though the maternal death rate is still low, pregnant women, especially those with comorbidities, are more likely to experience serious sickness. The review emphasizes how the COVID-19 vaccine affects menstrual cycles, showing brief, modest modifications without serious health hazards. Also included are the psychological effects of family planning choices during the pandemic. In conclusion, this narrative review offers a thorough assessment of the complicated and changing effects of SARS-CoV-2 on sexual and reproductive health. The different requirements of people and couples during and after the pandemic are highlighted, underscoring the necessity for ongoing study and specialized healthcare practices.",38738039,The Multi-faceted Effects of COVID-19 on Female Reproductive Health: An Updated Narrative Review.,2024-04-09,"Copyright © 2024, Al-Bdairi et al.","The Multi-faceted Effects of COVID-19 on Female Reproductive Health: An Updated Narrative Review. Concerns about the SARS-CoV-2 pandemic's possible impact on sexual and reproductive health have grown significantly. In this narrative review, the latest research on the effects of SARS-CoV-2 infection on several facets of sexual and reproductive health is reviewed. The review starts initially by going through the possible consequences of SARS-CoV-2 on female menstrual cycles. The virus may interfere with the menstrual cycle, which could affect hormone synthesis and endometrial reactions, according to newly available evidence. Menstrual blood loss may be impacted by COVID-19's potential to influence endothelial cell function and systemic hemostasis. Hypothalamic amenorrhea may be brought on by severe COVID-19 disease. There is little research on this subject, although most women resume their regular menstrual cycles after 1-2 months of recuperation. The review also examines how SARS-CoV-2 infection may affect assisted reproductive technology (ART) treatments. There are few clinical data, although some research points to potential effects on embryo quality. Overall, ART results, however, did not materially change from the time before the epidemic. Obstetric problems are more likely when SARS-CoV-2 infection occurs during pregnancy, especially in the third trimester. Even though the maternal death rate is still low, pregnant women, especially those with comorbidities, are more likely to experience serious sickness. The review 

doing the embedding

In [3]:
from langchain_community.llms import Ollama
from langchain.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import DataFrameLoader
from langchain_core.documents import Document
import requests, json

protocol = "https"
hostname = "chat.cosy.bio"


host = f"{protocol}://{hostname}"

account = {'email': 'melanie.altmann@studium.uni-hamburg.de', 'password': 'be$oPe96'}
auth_url = f"{host}/api/v1/auths/signin"
api_url = f"{host}/ollama"

auth_response = requests.post(auth_url, json=account)
jwt= json.loads(auth_response.text)["token"]

ollama_llm = Ollama(base_url=api_url, model="llama3", temperature = 0.0, headers= {"Authorization": "Bearer " + jwt})
ollama_embeddings = OllamaEmbeddings(base_url=api_url, model='nomic-embed-text', headers= {"Authorization": "Bearer " + jwt})

loader = DataFrameLoader(authors_df, page_content_column="research")
documents= loader.load()

In [4]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
    documents,embedding=ollama_embeddings,
)

#Perform a similarity search
query = authors_df["uid"]
results = vectorstore.similarity_search_with_score(query)

results

[(Document(page_content='Nanobody repertoire generated against the spike protein of ancestral SARS-CoV-2 remains efficacious against the rapidly evolving virus. To date, all major modes of monoclonal antibody therapy targeting SARS-CoV-2 have lost significant efficacy against the latest circulating variants. As SARS-CoV-2 omicron sublineages account for over 90% of COVID-19 infections, evasion of immune responses generated by vaccination or exposure to previous variants poses a significant challenge. A compelling new therapeutic strategy against SARS-CoV-2 is that of single-domain antibodies, termed nanobodies, which address certain limitations of monoclonal antibodies. Here, we demonstrate that our high-affinity nanobody repertoire, generated against wild-type SARS-CoV-2 spike protein (Mast et al., 2021), remains effective against variants of concern, including omicron BA.4/BA.5; a subset is predicted to counter resistance in emerging XBB and BQ.1.1 sublineages. Furthermore, we reveal

Store result in vector database (FAISS):

In [5]:
from langchain_community.vectorstores import FAISS
import faiss
import numpy as np


vector_db = FAISS.from_documents(loader.load(), ollama_embeddings)
vector_db.save_local("test_net_embedding")
#vector_db = FAISS.load_local("test_net_embedding", ollama_embeddings, allow_dangerous_deserialization=True)
vectors = vector_db.index.reconstruct_n(0, vector_db.index.ntotal)

compute pairwise semantic similarity aka create semantic similarity matrix:

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(vectors)

#Create a DataFrame for the similarity matrix
similarity_df = pd.DataFrame(similarity_matrix, index=authors_df.index, columns=authors_df.index)
similarity_df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1.000000,0.775573,1.000000,1.000000,0.775573,0.744601,0.744601,0.775573,1.000000,1.000000,...,0.775573,1.000000,0.775573,1.000000,0.775573,1.000000,0.775573,0.744601,0.744601,0.744601
1,0.775573,1.000000,0.775573,0.775573,1.000000,0.763069,0.763069,1.000000,0.775573,0.775573,...,1.000000,0.775573,1.000000,0.775573,1.000000,0.775573,1.000000,0.763069,0.763069,0.763069
2,1.000000,0.775573,1.000000,1.000000,0.775573,0.744601,0.744601,0.775573,1.000000,1.000000,...,0.775573,1.000000,0.775573,1.000000,0.775573,1.000000,0.775573,0.744601,0.744601,0.744601
3,1.000000,0.775573,1.000000,1.000000,0.775573,0.744601,0.744601,0.775573,1.000000,1.000000,...,0.775573,1.000000,0.775573,1.000000,0.775573,1.000000,0.775573,0.744601,0.744601,0.744601
4,0.775573,1.000000,0.775573,0.775573,1.000000,0.763069,0.763069,1.000000,0.775573,0.775573,...,1.000000,0.775573,1.000000,0.775573,1.000000,0.775573,1.000000,0.763069,0.763069,0.763069
5,0.744601,0.763069,0.744601,0.744601,0.763069,1.000000,1.000000,0.763069,0.744601,0.744601,...,0.763069,0.744601,0.763069,0.744601,0.763069,0.744601,0.763069,1.000000,1.000000,1.000000
6,0.744601,0.763069,0.744601,0.744601,0.763069,1.000000,1.000000,0.763069,0.744601,0.744601,...,0.763069,0.744601,0.763069,0.744601,0.763069,0.744601,0.763069,1.000000,1.000000,1.000000
7,0.775573,1.000000,0.775573,0.775573,1.000000,0.763069,0.763069,1.000000,0.775573,0.775573,...,1.000000,0.775573,1.000000,0.775573,1.000000,0.775573,1.000000,0.763069,0.763069,0.763069
8,1.000000,0.775573,1.000000,1.000000,0.775573,0.744601,0.744601,0.775573,1.000000,1.000000,...,0.775573,1.000000,0.775573,1.000000,0.775573,1.000000,0.775573,0.744601,0.744601,0.744601
9,1.000000,0.775573,1.000000,1.000000,0.775573,0.744601,0.744601,0.775573,1.000000,1.000000,...,0.775573,1.000000,0.775573,1.000000,0.775573,1.000000,0.775573,0.744601,0.744601,0.744601


In [7]:
# Assuming similarity_df and authors_df are already defined and contain the necessary data
from sklearn.cluster import DBSCAN

# Convert the similarity matrix to a distance matrix
distance_matrix = 1 - similarity_matrix

# Apply DBSCAN
dbscan = DBSCAN(metric='precomputed', eps=0.1, min_samples=2)
dbscan.fit(distance_matrix)

# Extract the cluster labels
cluster_labels = dbscan.labels_

# Add the cluster labels to the original DataFrame
authors_df['Cluster'] = cluster_labels

# Display the DataFrame with the cluster labels
authors_df

,forename,lastname,affiliation,pubmedid,country,city,latitude,longitude,page_content,uid,Title,Published,Copyright Information,research,Cluster
0,Adnan A,Al-Bdairi,"Infertility, Assisted Reproductive Technologies, Teba IVF and Genetic Center, Babylon, IRQ.",38738039,Iraq,Babylon,32.544713,44.431814,"Concerns about the SARS-CoV-2 pandemic's possible impact on sexual and reproductive health have grown significantly. In this narrative review, the latest research on the effects of SARS-CoV-2 infection on several facets of sexual and reproductive health is reviewed. The review starts initially by going through the possible consequences of SARS-CoV-2 on female menstrual cycles. The virus may interfere with the menstrual cycle, which could affect hormone synthesis and endometrial reactions, according to newly available evidence. Menstrual blood loss may be impacted by COVID-19's potential to influence endothelial cell function and systemic hemostasis. Hypothalamic amenorrhea may be brought on by severe COVID-19 disease. There is little research on this subject, although most women resume their regular menstrual cycles after 1-2 months of recuperation. The review also examines how SARS-CoV-2 infection may affect assisted reproductive technology (ART) treatments. There are few clinical data, although some research points to potential effects on embryo quality. Overall, ART results, however, did not materially change from the time before the epidemic. Obstetric problems are more likely when SARS-CoV-2 infection occurs during pregnancy, especially in the third trimester. Even though the maternal death rate is still low, pregnant women, especially those with comorbidities, are more likely to experience serious sickness. The review emphasizes how the COVID-19 vaccine affects menstrual cycles, showing brief, modest modifications without serious health hazards. Also included are the psychological effects of family planning choices during the pandemic. In conclusion, this narrative review offers a thorough assessment of the complicated and changing effects of SARS-CoV-2 on sexual and reproductive health. The different requirements of people and couples during and after the pandemic are highlighted, underscoring the necessity for ongoing study and specialized healthcare practices.",38738039,The Multi-faceted Effects of COVID-19 on Female Reproductive Health: An Updated Narrative Review.,2024-04-09,"Copyright © 2024, Al-Bdairi et al.","The Multi-faceted Effects of COVID-19 on Female Reproductive Health: An Updated Narrative Review. Concerns about the SARS-CoV-2 pandemic's possible impact on sexual and reproductive health have grown significantly. In this narrative review, the latest research on the effects of SARS-CoV-2 infection on several facets of sexual and reproductive health is reviewed. The review starts initially by going through the possible consequences of SARS-CoV-2 on female menstrual cycles. The virus may interfere with the menstrual cycle, which could affect hormone synthesis and endometrial reactions, according to newly available evidence. Menstrual blood loss may be impacted by COVID-19's potential to influence endothelial cell function and systemic hemostasis. Hypothalamic amenorrhea may be brought on by severe COVID-19 disease. There is little research on this subject, although most women resume their regular menstrual cycles after 1-2 months of recuperation. The review also examines how SARS-CoV-2 infection may affect assisted reproductive technology (ART) treatments. There are few clinical data, although some research points to potential effects on embryo quality. Overall, ART results, however, did not materially change from the time before the epidemic. Obstetric problems are more likely when SARS-CoV-2 infection occurs during pregnancy, especially in the third trimester. Even though the maternal death rate is still low, pregnant women, especially those with comorbidities, are more likely to experience serious sickness. The

In [9]:
from pyvis.network import Network
import pandas as pd
import seaborn as sns

paper_main_authors = pd.read_pickle("./paper_main_authors.pkl")

# Convert the similarity DataFrame to float64
similarity_df = similarity_df.astype('float64')

# Define a color palette for clusters
num_clusters = authors_df['Cluster'].nunique()
palette = sns.color_palette("Spectral", num_clusters)
cluster_colors = {cluster: f'#{int(color[0]*255):02x}{int(color[1]*255):02x}{int(color[2]*255):02x}' 
                  for cluster, color in zip(authors_df['Cluster'].unique(), palette)}

# Initialize the network
net = Network(height="750px", width="100%", bgcolor="#dddddd", font_color="black", filter_menu=True, notebook=True, cdn_resources='remote')
net.toggle_hide_edges_on_drag(False)
net.barnes_hut()

# Create a set of full names for first and last authors from paper_main_authors for quick lookup
highlight_authors = set(paper_main_authors['first_author']).union(set(paper_main_authors['last_author']))

# Add nodes with custom hover information
for idx, row in authors_df.iterrows():
    node = idx
    full_name = f"{row['forename']} {row['lastname']}"
    title = f"{full_name}: {row['affiliation']}"
    color = cluster_colors[row['Cluster']]
    
    # Check if the full name is in the highlight_authors set
    if full_name in highlight_authors:
        size = 50  # Highlighted node size
    else:
        size = 20  # Default node size
        
    net.add_node(node, label=full_name, title=title, color=color, size=size)

# Define a uniform color for all edges
edge_color = '#a6a6a6'  # Grey color for all edges

# Add edges with similarity values as hover information
for i, source in enumerate(similarity_df.index):
    for j, target in enumerate(similarity_df.columns):
        if i < j:  # Ensure each pair is considered only once
            similarity = similarity_df.loc[source, target]
            if similarity == 1:  # Only add edges with similarity of 1
                source_name = f"{authors_df.loc[source, 'forename']} {authors_df.loc[source, 'lastname']}"
                target_name = f"{authors_df.loc[target, 'forename']} {authors_df.loc[target, 'lastname']}"
                edge_title = f"Edge from {source_name} to {target_name}. Similarity: {similarity:.2f}"
                net.add_edge(source, target, title=edge_title, value=similarity, color=edge_color)

# Save and show the network
net.show("ex3.html")


ex3.html
